# "Predicción de la diferencia de goles y análisis de la ventaja de localía en partidos internacionales de fútbol"
Ciencia de Datos
Profesor: JESUS ANTONIO VILLARRAGA PALOMINO
Integrantes:
Karol Andrea Daza Rojas
Cristian Camilo Berbeo Villalobos
Daniel Felipe Fonseca Cabezas

# "Resumen"
Este cuadernillo documenta el análisis exploratorio, la limpieza y la preparación de datos, la ingeniería de variables, la construcción y evaluación de un modelo de regresión para predecir la diferencia de goles (goal_diff = home_score - away_score) y el análisis de la ventaja de localía usando la base de datos International Football Results (1872–2023) (archivo results.csv). Se incluyen las decisiones de modelado, gráficos y conclusiones.

# "Descripción de la base de datos"
Nombre del archivo: results.csv
Fuente: Conjunto público clásico que registra resultados internacionales desde 1872 hasta 2023 (Kaggle). 

El archivo previsto contiene registros de partidos con las siguientes columnas principales:
______________________________________________________________
|    Datos   |                  Descripcion                  |
--------------------------------------------------------------
|    date    |         Fecha del partido (YYYY-MM-DD)        |
--------------------------------------------------------------
|  home_team |                  Equipo local                 |
--------------------------------------------------------------
|  away_team |                Equipo visitante               |
--------------------------------------------------------------
| home_score |             Goles del equipo local            |
--------------------------------------------------------------
| away_score |           Goles del equipo visitante          |
--------------------------------------------------------------
| tournament |Tipo de competencia (Friendly, World Cup, etc.)|
--------------------------------------------------------------
|    city    |              Ciudad donde se jugó             |
--------------------------------------------------------------
|   country  |               País donde se jugó              |
--------------------------------------------------------------
|   neutral  | Booleano que indica si fue en cancha neutral  |
--------------------------------------------------------------

Observaciones:

- El dataset es amplio y abarca más de 48.000 partidos.

- Algunas inconsistencias posibles: variaciones en el nombre de torneos, nombres de equipos, y fechas antiguas. Se realizará limpieza y normalización.

# Objetivo General
evaluar un modelo de regresión que prediga la diferencia de goles (goal_diff) en partidos internacionales y cuantifique el efecto de la localía.
# Objetivos Especificos
1. Generar variables derivadas (ingeniería de features) relevantes para la predicción.
2. Entrenar un modelo de regresión (lineal y comparativo con otros) y evaluar su desempeño.
3. Analizar residuales y extraer conclusiones deportivas y estadísticas.